## 3D ERT arrays

In [1]:
import pygimli as pg
import pygimli.meshtools as mt
from pygimli.physics import ert
%matplotlib notebook

In [2]:
scheme = pg.DataContainerERT()
scheme.createSensor([0, 0, 0])
scheme.createSensor([1, 0, 0])
scheme.createSensor([2, 0, 0])
scheme.createSensor([3, 0, 0])
scheme.createSensor([0, 1, 0])
scheme.createSensor([1, 1, 0])
scheme.createSensor([2, 1, 0])
scheme.createSensor([3, 1, 0])
scheme.createFourPointData(scheme.size(), 0, 1, 2, 3)
scheme.createFourPointData(scheme.size(), 0, 1, 4, 5)
scheme.createFourPointData(scheme.size(), 0, 4, 1, 5)
scheme.createFourPointData(scheme.size(), 0, 4, 2, 6)
scheme.createFourPointData(scheme.size(), 0, 4, 3, 7)
scheme["k"] = ert.geometricFactors(scheme, dim=3)
print(scheme)
print(scheme["k"])

Data: Sensors: 8 data: 5, nonzero entries: ['a', 'b', 'k', 'm', 'n', 'valid']
5 [-18.849555921538762, 10.72606824533795, 10.72606824533795, 59.51518546006079, 183.65907399985898]


In [3]:
world = mt.createWorld(start=[-2, -2, -5], end=[5, 3, 0], worldMarker=True)
pl, _ = pg.show(world)

In [4]:
for sen in scheme.sensors():
    world.createNode(sen, marker=-99)
    
mesh = mt.createMesh(world, area=0.01)
pg.show(mesh, style="wireframe")

(<pyvista.plotting.plotting.Plotter at 0x1b721efedc0>, None)

In [5]:
fop = ert.ERTModelling(verbose=False)
fop.setData(scheme)
fop.setMesh(mesh)
model = pg.Vector(mesh.cellCount(), 1.0)
fop.createJacobian(model)

09/12/22 - 09:37:56 - pyGIMLi - INFO - Found 1 regions.
09/12/22 - 09:37:56 - pyGIMLi - INFO - Creating forward mesh from region infos.
09/12/22 - 09:38:03 - pyGIMLi - INFO - Creating refined mesh (P2) to solve forward task.


In [11]:
ind = 4
sens = fop.jacobian()[ind]
normsens = pg.utils.logDropTol(sens/mesh.cellSizes(), 1e-2)
mesh["sens"] = normsens / max(normsens)
mesh["asens"] = pg.abs(mesh["sens"])
pg.show(mesh, label="sens", cMap="bwr", filter={"threshold": dict(value=0.01, scalars="asens")})

(<pyvista.plotting.plotting.Plotter at 0x1b93db630a0>, None)